# <center> Inverse theory in imaging </center>

## <center> Michael Lamoureux </center>

#### <center> CMS Winter 2018, Vancouver </center>


## Goal of seismic imaging
- find and produce oil and gas
- find and produce more oil and gas
- economically, legally

## Goal of medical imaging
- heal the patient
- quickly, economically, legally/ethically

## Evolution of sub-goals (seismic)
1.  Detect anomalies
2. Locate anomalies
3. Image geological formations
4. Image and identify lithography
5. Image and identify fluids

## Evolution of algorithms
1. Plotting raw seismic data
2. Hand tracing of hyperbolas
3. Hand computation of time/space conversions, moveout
4. Signal processing, automation
5. Mathematical modeling, inverse theory

## Evolution of sensing equipment
1. Analog geophones (magnet/coil), tape recording
2. Digital geophones (magnet/coil, 24bit ADC), digital recording
3. 3C geophones + hydrophones
4. 3C MEMS accelerometers
5. DAS fibre optic sensors 
6. Dynamite, thumpers, Vibroseis sources

## Evolution of models
1. 1D seismic reflectivity
2. Ray tracing, Kirchoff migration
2. 2D, 3D wave equations
3. Acoustic wave equation, 1 or 2 variable coefficients
3. Elastic wave equation
4. Visco-elastic wave equation, with fluids

## Processing flow (from AAPG)
1. Demultiplex
2. Edit
3. Geometry
4. Antialias filter
5. Gain recovery
6. Deconvolution
7. Statics
8. Demultiple
9. f–k or apparent velocity filter
10. Normal moveout (NMO) correction
11. Dip moveout (DMO) correction
12. Common midpoint (CMP) stack
13. Poststack filter
14. Poststack mix

## Processing flow in full waveform inversion (SLIM/UBC)

<center><img src="FWI_schematic.png" width="1000"  /></center>

## <center> Examples: FD simulation  of wave equation</center>

A wave packet travelling from high velocity area, to low velocity .

Transmitted wave has a shorter wavelength, different angle of attack. 

The angles are related by Snell's law. 

In [3]:
%%writefile myFile1.html
<!DOCTYPE html>
<html><head id="Barebones">
        <meta charset="UTF-8" />
        <meta name="viewport" content="width=device-width, initial-scale=1.0" />
        <script src="GPGPUtility.js"></script>
        <script src="WaveEqn.js"></script>
        <script src="WavePlot.js"></script>
        <title>WebGL wave example</title>
        </head>
        <body>
        <button id="startbutton">Restart</button> <p>
        <div class="content">
            <h1>Snell's law: a refracting wave packet. Click "Restart" for animation</h1>


            <figure class="center" id="results">
                <figcaption>
                    You should see a waveform traveling here. 
                </figcaption>
            </figure>
        </div>

        <br class="clear" />
        <script>
        startbutton.addEventListener('click', function (ev) {
            location.reload();
            ev.preventDefault();
        }, false);
        </script>
        
        <script>
            "use strict";

            /** The canvas onto which we render the wave function & potential */
            var canvas;
            var dt;
            var gl;
            var gpgpUtility;
            var nframes;
            var nsteps;
            var parent;
            var renderer;
            var vel_top;
            var vel_bot;
            var wp;
            var wt;
            var wlen;
            var wang;
            var waveengine;
            var waveFunctionData, waveFunctionDataD, waveFunctionTexture0, waveFunctionTexture1, waveFunctionTexture2;
            var x0;
            var y0;
            var xLength;
            var xResolution;
            var yLength;
            var yResolution;


            // Physical parameters for the wave motion simulation
            
            xLength = 4000.0; // in meters
            yLength = 4000.0; // in meters
            vel_top = 1000.0; // velocity in m/s, in top half
            vel_bot = 500.0; // velocity in m/s, in bottom half
            x0 = .25*xLength; // location of initial Gaussian
            y0 = .75*yLength;
            wp = 0.05*xLength; // width of the initial Gaussian wave, propagating direction
            wt = 0.05*xLength; // width of the initial Gaussian wave, transverse direction
            wlen = .05*xLength; // wavelength of the propagating wave
            wang = -45; // propagation angle in degrees (0 is horizontal, to the right)

            // Parameters for the numerical simulation
            
            xResolution = 1000; // number of sample points (x-dir y-dir)
            yResolution = 1000; 
            dt = .001; // sample interval for time, in seconds
            nframes = 600; // Total number of frames to compute and display
            nsteps = 5; // Number of delta t timesteps per frams

            // Setup the GPU and canvas to diplay results
            
            gpgpUtility = new vizit.utility.GPGPUtility(xResolution, yResolution, {premultipliedAlpha: false});
            gpgpUtility.setProblemSize(xResolution, yResolution);

            canvas = gpgpUtility.getCanvas();
            canvas.style.height = yResolution + "px";
            parent = document.getElementById("results")
            parent.insertBefore(canvas, parent.firstChild);
            
            // an initial plane wave, constrained
            function f_plane(x,w,lambda) {
                return Math.exp(-(x*x)/(w*w))*Math.cos(2*Math.PI*x/lambda);
            }
            // the corresponding derivative
            function f_planeD(x,w,lambda) {
                return -Math.exp(-(x*x)/(w*w))*((2*x/(w*w))*Math.cos(2*Math.PI*x/lambda)+
                                                (2*Math.PI/lambda)*Math.sin(2*Math.PI*x/lambda));
            }
            // a window in the transverse direction
            function f_win(y,w) {
                return Math.exp(-(y*y)/(w*w));
            }
            // the corresponding derivative
            function f_winD(y,w) {
                return -Math.exp(-(y*y)/(w*w))*(2*y/(w*w));
            }
 
            // the initial waveform
            function gaussWaveform(waveform,xRez,yRez,xLen,yLen) {
                var aa = Math.cos(Math.PI*wang/180); // rotation matrix
                var bb = Math.sin(Math.PI*wang/180);
                for (var j = 0; j < yRez; ++j) {
                    for (var i = 0; i < xRez; ++i) {
                        var x = xLen*(i/xRez) - x0;
                        var y = yLen*(j/yRez) - y0;
                        waveform[4*(i + j*xRez)] = 
                             10*f_plane(aa*x + bb*y,wp,wlen)*f_win(-bb*x+aa*y,wt);
                      }
                }
                return waveform;
            }
            function gaussWaveformD(waveform,xRez,yRez,xLen,yLen) {
                var aa = Math.cos(Math.PI*wang/180); // rotation matrix
                var bb = Math.sin(Math.PI*wang/180);
                for (var j = 0; j < yRez; ++j) {
                    for (var i = 0; i < xRez; ++i) {
                        var x = xLen*(i/xRez) - x0;
                        var y = yLen*(j/yRez) - y0;
                        waveform[4*(i + j*xRez)] = 
                             10*f_plane(aa*x + bb*y,wp,wlen)*f_win(-bb*x+aa*y,wt)
                                - 10*dt*vel_top*f_planeD(aa*x + bb*y,wp,wlen)*f_win(-bb*x+aa*y,wt);
                      }
                }
                return waveform;
            }
            // we stuff velocity^2 info into the green channel;
            function velField(waveform,xRez,yRez) {
                for (var j = 0; j < yRez; ++j) {
                    for (var i = 0; i < xRez; ++i) {
                        waveform[4*(i + j*xRez) + 1] = 
                            (j > yRez/2) ? vel_top**2 : vel_bot**2;
                    }
                }
                return waveform;
            }

            /**
             * Run the simulation for n time steps, then show the results.
             */
            function nextFrame() {
                for (var i = 0; i < nsteps; ++i) {
                    waveengine.timestep();
                }

                renderer.show(waveengine.getRenderedTexture());

                if (nframes--) {
                    requestAnimationFrame(nextFrame);
                }
            }

            waveFunctionData = new Float32Array(4 * xResolution * yResolution);
            waveFunctionData = velField(waveFunctionData,xResolution,yResolution);
            waveFunctionData = gaussWaveform(waveFunctionData,xResolution,yResolution,xLength,yLength);
 
            waveFunctionDataD = new Float32Array(4 * xResolution * yResolution);
            waveFunctionDataD = velField(waveFunctionDataD,xResolution,yResolution);
            waveFunctionDataD = gaussWaveformD(waveFunctionDataD,xResolution,yResolution,xLength,yLength);
 
            waveFunctionTexture0 = gpgpUtility.makeTexture(WebGLRenderingContext.FLOAT, waveFunctionData);
            waveFunctionTexture1 = gpgpUtility.makeTexture(WebGLRenderingContext.FLOAT, waveFunctionDataD);
            waveFunctionTexture2 = gpgpUtility.makeTexture(WebGLRenderingContext.FLOAT, waveFunctionData);


            waveengine = new WaveEngine(gpgpUtility, xResolution, yResolution, xLength, yLength, dt);
            renderer = new WaveResults(gpgpUtility, parent, xResolution, yResolution);

            waveengine.setInitialTextures(waveFunctionTexture0, waveFunctionTexture1, waveFunctionTexture2);

            renderer.show(waveengine.getRenderedTexture());

            requestAnimationFrame(nextFrame);
    </script>
</html>

Overwriting myFile1.html


In [2]:
%%html 
<iframe src='myFile1.html' width=1000 height=1000>

Snell's law by geometry.  Wavelengths related by sines:
<center><img src="SnellsLaw3.png" width="1000"  /></center>

Wavelength, frequency  related by velocity,
$$\lambda f = v,$$ so
$$\frac{\lambda_2}{\sin \theta_2} = \frac{\lambda_1}{\sin \theta_1} \quad \mbox{ becomes } \quad
\frac{v_2/f}{\sin \theta_2} = \frac{v_1/f}{\sin \theta_1}.$$

Snell's law via ray paths (least time):

<center><img src="SnellsLaw1.png" width="1000"  /></center>

## FWI example (Zubov)

2D acoustic wave equation, with velocity the unknown parameter field 

Three images to watch
- estimated velocity 
- true velicity
- misfit of shot record 

In [2]:
%%HTML
<center><video width="1000"  controls> <source src="FWI_Zubov_Horz.mov"> </video> </center>

## New sensing mode: DAS (Hardeman-Voys)

<center><img src="DAS_Newell.png" width="1000"  /></center>

## Acknowledgements

- Students and PDF: Heather Hardeman-Voys, Da Li, Vladimir Zubov
- CREWES sponsors
- NSERC CRDPJ 461179-13 , Discovery grant

## <center> Thank you! </center>